In [3]:
import numpy as np
from sklearn import decomposition
from sklearn import linear_model
import matplotlib.pyplot as plt
from collections import OrderedDict
import cv2
import dlib
%matplotlib inline


In [4]:
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

# Check if a point is inside a rectangle
def rect_contains(rect, point):
    if point[0] < rect[0]:
        return False
    elif point[1] < rect[1]:
        return False
    elif point[0] > rect[2]:
        return False
    elif point[1] > rect[3]:
        return False
    return True

In [5]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('data/shape_predictor_68_face_landmarks.dat')

In [27]:
def get_landmarks(image):
    
    size = image.shape

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Locate face position
    rects = detector(gray, 1)    
    subdiv = cv2.Subdiv2D((0, 0, size[1], size[0]))

    delauay_points = []
    for rect in rects:
        shape = predictor(gray, rect)
        for p in shape.parts():
            delauay_points.append((p.x, p.y))
            subdiv.insert((p.x, p.y))
            

    triangleList = subdiv.getTriangleList()
    r = (0, 0, size[1], size[0])
    
    triangle_points = []
 
    for t in triangleList :
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
         
        if rect_contains(r, pt1) and rect_contains(r, pt2) and rect_contains(r, pt3) :
            triangle_points.append((pt1, pt2, pt3))
                         
    return delauay_points, triangle_points


In [7]:
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst


# Warps and alpha blends triangular regions from img1 and img2 to img
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []

    for i in xrange(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask



In [34]:
src = cv2.imread('./images/image.jpg')
src = src[:,:,::-1] # not effective but for rendering with matplotlib
src_size = (500, int(500 * src.shape[1] / src.shape[0]))
src = cv2.resize(src, src_size)
src_delauay_points, src_triangle_points = get_landmarks(src)

dst = cv2.imread('./images/image3.jpeg')
dst = dst[:,:,::-1] # not effective but for rendering with matplotlib
dst_size = (500, int(500 * dst.shape[1] / dst.shape[0]))
dst = cv2.resize(dst, dst_size)
dst_delauay_points, dst_triangle_points = get_landmarks(dst)

alpha = 0.5

points = []

for src_p, dst_p in zip(src_delauay_points, dst_delauay_points):
    x = src_p[0] * (1.0 - alpha) + dst_p[0] * alpha
    y = src_p[1] * (1.0 - alpha) + dst_p[1] * alpha
    points.append((x, y))
    
morphed_img = np.zeros(src.shape, dtype = src.dtype)


rectangles[[(218, 219) (373, 374)]]
rectangles[[(142, 118) (365, 341)]]


In [35]:
src_triangle_points

[((307.0, 295.0), (336.0, 307.0), (308.0, 311.0)),
 ((336.0, 307.0), (307.0, 295.0), (341.0, 291.0)),
 ((287.0, 319.0), (277.0, 341.0), (262.0, 343.0)),
 ((277.0, 341.0), (287.0, 319.0), (291.0, 338.0)),
 ((202.0, 285.0), (205.0, 263.0), (249.0, 262.0)),
 ((257.0, 268.0), (262.0, 343.0), (202.0, 309.0)),
 ((262.0, 343.0), (257.0, 268.0), (287.0, 319.0)),
 ((202.0, 309.0), (202.0, 285.0), (257.0, 268.0)),
 ((277.0, 341.0), (290.0, 345.0), (274.0, 352.0)),
 ((290.0, 345.0), (277.0, 341.0), (290.0, 344.0)),
 ((206.0, 331.0), (202.0, 309.0), (262.0, 343.0)),
 ((274.0, 352.0), (266.0, 389.0), (247.0, 379.0)),
 ((266.0, 389.0), (274.0, 352.0), (287.0, 356.0)),
 ((216.0, 351.0), (206.0, 331.0), (262.0, 343.0)),
 ((294.0, 356.0), (305.0, 370.0), (287.0, 356.0)),
 ((305.0, 370.0), (294.0, 356.0), (300.0, 354.0)),
 ((231.0, 367.0), (216.0, 351.0), (262.0, 343.0)),
 ((287.0, 356.0), (305.0, 370.0), (297.0, 385.0)),
 ((247.0, 379.0), (231.0, 367.0), (262.0, 343.0)),
 ((283.0, 391.0), (266.0, 389.0

In [15]:
draw_delaunay(image, subdiv)

plt.figure(figsize=(6, 6))
plt.imshow(image)

NameError: name 'draw_delaunay' is not defined